---

# Authenticator

---

In [1]:
import getpass

In [2]:
import safenet

Basic SAFE interface generated


In [3]:
myApp = safenet.app()

In [4]:
myAuth = myApp.authenticator

In [5]:
locator = myApp.ffi_app.new('char[]',getpass.getpass().encode('utf-8'))

········


In [6]:
pw = myApp.ffi_app.new('char[]',getpass.getpass().encode('utf-8'))

········


In [7]:
def printFileStem(one,two,filename):
    print(myApp.ffi_app.string(filename))

In [8]:
myAuth._auth_exe_file_stem(myApp.ffi_app.NULL,o_cb=printFileStem)

action succeeded
b'python3'


'gotResult'

In [9]:
myAuth._auth_set_additional_search_path(myApp.ffi_app.new('char[]',b'compiled_binaries/'),myApp.ffi_app.NULL)

action succeeded


'gotResult'

In [10]:
myAuth._login(locator,pw,myApp.ffi_app.NULL)

action succeeded


'gotResult'

---

# App

---

In [11]:
newPermissions,permission_addon=safenet.safeUtils.ContainerPermissions(b'_publicNames',ffi=myApp.ffi_app)

In [12]:
newPermissions,permission_addon=safenet.safeUtils.ContainerPermissions(b'_music',ffi=myApp.ffi_app)

In [13]:
myAuth_,addData=safenet.safeUtils.AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True,ffi=myApp.ffi_app)

In [14]:
encodedAuth = myApp._encode_auth_req(myAuth_,myApp.ffi_app.NULL)

action succeeded


---

this needs to become nicer

---

In [15]:
from cffi import FFI
ffi = FFI()

sysUri = FFI()
with open('safenet/extracted_headers/safe_app_datatype_declarations','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('safenet/extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('compiled_binaries/libsystem_uri.so')

import os

pathToHandler=(os.getcwd()+'/safenet/uriHandler.py').encode()

import base64

schemeName=b'safe-'+base64.b64encode(b'newProgId_unique_thing').strip(b'=')

@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)
    
bundle=sysUri.new('char[]',ffi.string(myAuth_.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth_.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth_.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


In [16]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


In [17]:
with open('safenet/answer','r') as f:
    answer=f.readlines()

In [18]:
import re
data=re.compile("'safe-.+?:(.+?)'")
my=data.findall(answer[-1])
msg = ffi.new('char[]',my[-1].encode())
returnDict={}
myHand=ffi.new_handle(returnDict)

---

this needs to become nicer

---

In [19]:
grantedAuth = myApp._decode_ipc_msg(msg,myHand)

In [20]:
appItself = myApp._app_registered(myAuth_.app.id,grantedAuth[0],myApp.ffi_app.NULL)

action succeeded


In [21]:
with open('safenet/myNewMutableData_as_bytes','rb') as f:
    readData=f.read()

In [22]:
dataOnSafe=safenet.safeUtils.getffiMutable(readData,myApp.ffi_app)

In [23]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataKey*, uint64_t)")
def result_mdata_list_keys(user_data, result, key, size):
    if result.error_code != 0:
        print(myApp.ffi_app.string(result.description))
    if size>0:
        print(myApp.ffi_app.string(key.key))

In [24]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataValue*, uint64_t)")
def result_mdata_list_values(user_data, result, value, size):
    if result.error_code != 0:
        print(ffi.string(result.description))
    if size>0:
        print(ffi.string(value.content))

In [25]:
myApp.lib.safe_app.mdata_list_keys(appItself,dataOnSafe,myApp.ffi_app.NULL,result_mdata_list_keys)

b'dask'


In [26]:
myApp.lib.safe_app.mdata_list_values(appItself,dataOnSafe,myApp.ffi_app.NULL,result_mdata_list_values)

b'we get closer...'


---
### why doesn't this work?
---

In [ ]:
myApp.mutableData._mdata_list_keys(appItself,dataOnSafe,myApp.ffi_app.NULL)

In [ ]:
myApp.mutableData._mdata_list_values(appItself,dataOnSafe,myApp.ffi_app.NULL)